<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/11.nlp/ExploreWordNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/11.nlp/ExploreWordNet.ipynb)

# Exploring WordNet

This notebook explores WordNet synsets, presenting a simple method for finding in a text all mentions of all hyponyms of a given node in the WordNet hierarchy (e.g., finding all *buildings* in a text).  Upload this notebook at the end of class.

In [1]:
import re

import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Get the synsets for a given word.  The synsets here are roughly ordered by frequency of use (in a small tagged dataset), so that more frequent senses occur first.

In [2]:
for synset in wn.synsets('speak'):
    print (synset, synset.definition())

Synset('talk.v.02') express in speech
Synset('talk.v.01') exchange thoughts; talk with
Synset('speak.v.03') use language
Synset('address.v.02') give a speech to
Synset('speak.v.05') make a characteristic or natural sound


Get the words/phrases in that synset.

In [3]:
for lemma in wn.synset("talk.v.01").lemmas():
    print (lemma.name())

talk
speak


In [4]:
# Functions from http://www.nltk.org/howto/wordnet.html to get *all* of a synset's hyponym/hypernyms
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()

Find all of the synsets that are hyponyms of the target synset (*descendents* in the WordNet hierarchy)

In [5]:
list(wn.synset("talk.v.02").closure(hypo))

[Synset('cackle.v.01'),
 Synset('chatter.v.05'),
 Synset('speak_up.v.02'),
 Synset('rasp.v.02'),
 Synset('tone.v.02'),
 Synset('generalize.v.02'),
 Synset('slur.v.03'),
 Synset('whisper.v.01'),
 Synset('yack.v.01'),
 Synset('babble.v.01'),
 Synset('rant.v.01'),
 Synset('hiss.v.03'),
 Synset('mumble.v.01'),
 Synset('bark.v.01'),
 Synset('verbalize.v.01'),
 Synset('drone.v.02'),
 Synset('read.v.03'),
 Synset('deliver.v.01'),
 Synset('gulp.v.02'),
 Synset('shout.v.01'),
 Synset('blubber.v.02'),
 Synset('blurt_out.v.01'),
 Synset('snivel.v.01'),
 Synset('whiff.v.05'),
 Synset('snap.v.01'),
 Synset('bumble.v.03'),
 Synset('sing.v.02'),
 Synset('murmur.v.01'),
 Synset('open_up.v.07'),
 Synset('chatter.v.04'),
 Synset('troll.v.07'),
 Synset('lip_off.v.01'),
 Synset('peep.v.04'),
 Synset('enthuse.v.02'),
 Synset('speak_in_tongues.v.01'),
 Synset('swallow.v.04'),
 Synset('tone.v.01'),
 Synset('begin.v.04'),
 Synset('talk_of.v.01'),
 Synset('bay.v.01'),
 Synset('vocalize.v.05'),
 Synset('call.v.

Find all of the synsets that are hyperyms (*ancestors* up the tree) of the target synset

In [6]:
list(wn.synset("communicate.v.02").closure(hyper))

[Synset('interact.v.01'), Synset('act.v.01')]

In [7]:
def get_words_in_hypo(synset):
    """ Returns a list of words/phrases that comprise the hyponyms of a synset.
    """
    words=set()
    hyponym_synsets=list(synset.closure(hypo))
    hyponym_synsets.append(synset)
    for synset in hyponym_synsets:
        for l in synset.lemmas():
            word=l.name()
            word=re.sub("_", " ", word)
            words.add(word)

    return words

In [8]:
get_words_in_hypo(wn.synset("color.n.01"))

{"Davy's gray",
 "Davy's grey",
 'Indian red',
 'Paris green',
 'Prussian blue',
 'Turkey red',
 'Tyrian purple',
 'Vandyke brown',
 'Venetian red',
 'achromasia',
 'achromatic color',
 'achromatic colour',
 'alabaster',
 'alizarine red',
 'amber',
 'apatetic coloration',
 'aposematic coloration',
 'apricot',
 'aqua',
 'aquamarine',
 'ash gray',
 'ash grey',
 'azure',
 'beige',
 'black',
 'blackness',
 'bleach',
 'blond',
 'blonde',
 'blondness',
 'blue',
 'blue green',
 'blueness',
 'bluish green',
 'bone',
 'bottle green',
 'brick red',
 'brown',
 'brownish yellow',
 'brownness',
 'buff',
 'burgundy',
 'burnt sienna',
 'burnt umber',
 'canary',
 'canary yellow',
 'caramel',
 'caramel brown',
 'cardinal',
 'carmine',
 'carnation',
 'cerise',
 'cerulean',
 'chalk',
 'charcoal',
 'charcoal gray',
 'charcoal grey',
 'chartreuse',
 'cherry',
 'cherry red',
 'chestnut',
 'chocolate',
 'chromatic color',
 'chromatic colour',
 'chromatism',
 'chrome green',
 'chrome red',
 'claret',
 'coal b

In [9]:
def find_all_words_in_text(words, spacy_tokens):
    """ For a given set of words, find each instance among a list of tokens already
    processed by spacy.  Returns a list of token indexes that match.  (Note this only
    identifies single words, not multi-word phrases.)
    """
    all_matches=[]
    for idx, token in enumerate(spacy_tokens):
        if token.lemma_ in words:
            all_matches.append(idx)
    return all_matches

In [10]:
def print_concordance(matches, spacy_tokens, window=3):
    """ For a given set of token indexes, prints out a window of words around each match,
    in the style of a concordance.
    """

    RED="\x1b[31m"
    BLACK="\x1b[0m"

    spacing=window*10
    for match in matches:
        start=match-window
        end=match+window+1
        if start < 0:
            start=0
        if end > len(spacy_tokens):
            end=len(spacy_tokens)
        pre=' '.join([token.text for token in spacy_tokens[start:match]])
        post=' '.join([token.text for token in spacy_tokens[match+1:end]])
        print("%s %s%s%s %s" % (pre.rjust(spacing), RED, spacy_tokens[match].text, BLACK, post))

In [11]:
def read_text(filename):
    """ Read a text, replacing all whitespace sequences with a single space.
    """
    with open(filename, encoding="utf-8") as file:
        return re.sub(r"\s+", " ", file.read())

In [12]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt

--2025-11-05 00:40:26--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691804 (676K) [text/plain]
Saving to: ‘1342_pride_and_prejudice.txt’

1342_pride_and_prej 100%[===================>] 675.59K  --.-KB/s    in 0.04s   

2025-11-05 00:40:27 (15.1 MB/s) - ‘1342_pride_and_prejudice.txt’ saved [691804/691804]



In [13]:
book = read_text("1342_pride_and_prejudice.txt")

In [14]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['ner,parser'])
nlp.remove_pipe('ner')
nlp.remove_pipe('parser');

In [15]:
spacy_tokens = nlp(book)

In [16]:
def wordnet_search(synset, spacy_tokens):
    """ This functions searchs through all of the tokens in the spacy_tokens argument to find
    any mention of words in the synset or any of its hyponyms.
    """
    targets = get_words_in_hypo(synset)
    matches = find_all_words_in_text(targets, spacy_tokens)
    print_concordance(matches, spacy_tokens)

**Q1.** Let's do a very coarse tagging of a document to find all of the mentions of a specific WordNet synset and all of its hyponyms. Using the functions above, find all of the color terms in *Pride and Prejudice*.

In [17]:
wordnet_search(wn.synset("color.n.01"), spacy_tokens)

                     he wore a blue coat , and
                    and rode a black horse . An
                   a bottle of wine a day .
                     I liked a red coat myself very
                  given to her complexion , and doubt
              till summoned to coffee . She was
                 walking , the tone of her voice
                   with a fine complexion and good -
                   , but their colour and shape ,
             Nicholls has made white soup enough ,
                        is _ a shade in a character
            reject the offered olive - branch .
                   idea of the olive - branch perhaps
                     come in a scarlet coat , and
                  in any other colour . As for
                 In a softened tone she declared herself
                . Both changed colour , one looked
                  , one looked white , the other
                   , the other red . Mr. Wickham
              , breathing port wine , who follo

**Q2.** Find all of the vehicles mentioned in *Pride and Prejudice*.

In [18]:
wordnet_search(wn.synset("vehicle.n.01"), spacy_tokens)

                   Monday in a chaise and four to
                    not keep a carriage , and had
                     ball in a hack chaise . ”
                     in a hack chaise . ” “
                    I have the carriage ? ” said
                Mr. Bingley 's chaise to go to
                     go in the coach . ” “
                could have the carriage . ” Elizabeth
                  , though the carriage was not to
               offered her the carriage , and she
                  offer of the chaise to an invitation
        afterwards ordered her carriage . Upon this
                     to give a flat denial , and
                in general and ordinary cases between friend
                  beg that the carriage might be sent
             possibly have the carriage before Tuesday ;
                Mr. Bingley 's carriage immediately , and
                 in her little phaeton and ponies .
                 be adapted to ordinary occasions , I
                  as th

**Q3.** Find all of the verbs of speaking in *Pride and Prejudice*.

In [27]:
wordnet_search(wn.synset("speak.v.01"), spacy_tokens)

                    that he is considered the rightful property
                   and four to see the place ,
                    to see the place , and was
                   how can you talk so ! But
                         ” “ I see no occasion for
                 indeed go and see Mr. Bingley when
                       ” “ But consider your daughters .
                  very glad to see you ; and
               take delight in vexing me . You
                   and live to see many young men
                 , he suddenly addressed her with :
             contain herself , began scolding one of
                      “ Do you consider the forms of
                    know , and read great books and
                would not have called on him .
                    over , she began to declare that
                       , as he spoke , he left
                     I can but see one of my
                      ; but he saw only the father
                     ; and she began to fe

**Q4.** Find all of the people in *Pride and Prejudice*.

In [35]:
wn.synsets("had")

[Synset('have.v.01'),
 Synset('have.v.02'),
 Synset('experience.v.03'),
 Synset('own.v.01'),
 Synset('get.v.03'),
 Synset('consume.v.02'),
 Synset('have.v.07'),
 Synset('hold.v.03'),
 Synset('have.v.09'),
 Synset('have.v.10'),
 Synset('have.v.11'),
 Synset('have.v.12'),
 Synset('induce.v.02'),
 Synset('accept.v.02'),
 Synset('receive.v.01'),
 Synset('suffer.v.02'),
 Synset('have.v.17'),
 Synset('give_birth.v.01'),
 Synset('take.v.35')]

In [37]:
for synset in wn.synsets("had"):
  print(synset.definition())

have or possess, either in a concrete or an abstract sense
have as a feature
go through (mental or physical states or experiences)
have ownership or possession of
cause to move; cause to be in a certain position or condition
serve oneself to, or consume regularly
have a personal or business relationship with someone
organize or be responsible for
have left
be confronted with
undergo
suffer from; be ill with
cause to do; cause to act in a specified manner
receive willingly something given or offered
get something; come into possession of
undergo (as of injuries and illnesses)
achieve a point or goal
cause to be born
have sex with; archaic use


In [34]:
wn.synsets("person")

[Synset('person.n.01'), Synset('person.n.02'), Synset('person.n.03')]

In [31]:
wordnet_search(wn.synset("person.n.01"), spacy_tokens)

Streaming output truncated to the last 5000 lines.
                        . “ We have not determined how
               No scheme could have been more agreeable
                       Oh , my dear , dear aunt
                     my dear , dear aunt , ”
                   dear , dear aunt , ” she
                 give me fresh life and vigour .
                What are young men to rocks and
                  young men to rocks and mountains ?
                 be like other travellers , without being
                 know where we have gone -- we
             recollect what we have seen . Lakes
          quarreling about its relative situation . Let
             the generality of travellers . ” Chapter
                     ; for she had seen her sister
                  had seen her sister looking so well
                     , and the prospect of her northern
                was a constant source of delight .
                  all that she had heard of its
                  heard of 

**Q5.** The methods above all identify *any* mentions of a WordNet synset in a text  -- e.g., every instance of *bank* would be identified as a hit for query bank.n.01 ("sloping land ..."), even if its specific word sense in context was the financial institution (or even a verb).

How might we improve on this method?